## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-16-46-55 +0000,bbc,Vinted users furious over InPost UK parcel delays,https://www.bbc.com/news/articles/c4gvpgy1w20o...
1,2025-09-25-16-46-33 +0000,nyt,Justice Dept. Official Pushes Prosecutors to I...,https://www.nytimes.com/2025/09/25/us/politics...
2,2025-09-25-16-41-36 +0000,nyt,Wyden Presses JPMorgan Chief Dimon for Answers...,https://www.nytimes.com/2025/09/25/us/politics...
3,2025-09-25-16-41-35 +0000,cbc,Ottawa to share details about Canada Post’s fa...,https://www.cbc.ca/news/politics/livestory/ott...
4,2025-09-25-16-41-00 +0000,nypost,Bloodthirsty squirrel terrorizes Calif. neighb...,https://nypost.com/2025/09/25/us-news/bloodthi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2350/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
116,trump,46
238,ice,17
90,president,16
232,dallas,14
389,shooting,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
81,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...,115
14,2025-09-25-16-25-34 +0000,nypost,Disney gears up for court battle with Trump af...,https://nypost.com/2025/09/25/business/disney-...,95
282,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,91
34,2025-09-25-15-52-04 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...,87
255,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,86


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
81,115,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...
269,77,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
17,58,2025-09-25-16-17-15 +0000,nypost,Over 100 ex-ABC journalists urge Disney’s Bob ...,https://nypost.com/2025/09/25/business/over-10...
161,53,2025-09-25-09-32-15 +0000,nypost,Former French President Sarkozy found guilty o...,https://nypost.com/2025/09/25/world-news/forme...
208,47,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
34,41,2025-09-25-15-52-04 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...
41,38,2025-09-25-15-30-26 +0000,nypost,Minivan driver charged after German tourist ki...,https://nypost.com/2025/09/25/us-news/minivan-...
124,35,2025-09-25-11-47-00 +0000,wsj,The Office of Management and Budget told feder...,https://www.wsj.com/politics/policy/white-hous...
139,32,2025-09-25-10-49-00 +0000,wsj,Denmark Says New Drone Flights Over Military B...,https://www.wsj.com/world/europe/denmark-says-...
45,31,2025-09-25-15-21-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
